In [1]:
from google.colab import drive
drive.mount('/content/drive')

import zipfile

zip_file_path = '/content/drive/MyDrive/turkiye-is-bankasi-ml-challenge-5.zip'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall()
zip_ref.close()

Mounted at /content/drive


In [4]:
!pip install fastparquet
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.0 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from fastparquet import ParquetFile
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier, PassiveAggressiveClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [6]:
pf = ParquetFile("/content/train_final.parquet")
pf1 = ParquetFile("/content/test_final.parquet")
pf2 = ParquetFile("/content/submission_sample_final.parquet")

submit = pf2.to_pandas()
train = pf.to_pandas()
test = pf1.to_pandas()

top_n = 15

top_carriers = train["carrier"].value_counts().head(top_n).index.tolist()

train["carrier_encoded"] = train["carrier"].apply(lambda x: x if x in top_carriers else "Other")

label_encoder = LabelEncoder()
train["carrier_encoded"] = label_encoder.fit_transform(train["carrier_encoded"])
top_devicebrand = train["devicebrand"].value_counts().head(20).index.tolist()

train["devicebrand_encoded"] = train["devicebrand"].apply(lambda x: x if x in top_devicebrand else "Other")

label_encoder = LabelEncoder()
train["devicebrand_encoded"] = label_encoder.fit_transform(train["devicebrand_encoded"])

train = train.drop(columns=['id'])
train = train.drop(columns=['carrier'])
train = train.drop(columns=['devicebrand'])
train[['Ilk_menu', 'Ikinci_menu', 'Ucuncu_menu']] = train['target'].str.split(',', expand=True)
train = train.drop(columns=['target'])

top_carriers = test["carrier"].value_counts().head(15).index.tolist()

test["carrier_encoded"] = test["carrier"].apply(lambda x: x if x in top_carriers else "Other")

label_encoder = LabelEncoder()
test["carrier_encoded"] = label_encoder.fit_transform(test["carrier_encoded"])

top_carriers = test["devicebrand"].value_counts().head(20).index.tolist()

test["devicebrand_encoded"] = test["devicebrand"].apply(lambda x: x if x in top_carriers else "Other")

label_encoder = LabelEncoder()
test["devicebrand_encoded"] = label_encoder.fit_transform(test["devicebrand_encoded"])

test = test.drop(columns=['id'])
test = test.drop(columns=['carrier'])
test = test.drop(columns=['devicebrand'])

menu_mapping1 = {
    ' menu1': 1,
    ' menu2': 2,
    ' menu3': 3,
    ' menu4': 4,
    ' menu5': 5,
    ' menu6': 6,
    ' menu7': 7,
    ' menu8': 8,
    ' menu9': 9,
}
menu_mapping = {
    'menu1': 1,
    'menu2': 2,
    'menu3': 3,
    'menu4': 4,
    'menu5': 5,
    'menu6': 6,
    'menu7': 7,
    'menu8': 8,
    'menu9': 9,
}
train['Ilk_menu'] = train['Ilk_menu'].replace(menu_mapping)
train['Ikinci_menu'] = train['Ikinci_menu'].replace(menu_mapping1)
train['Ucuncu_menu'] = train['Ucuncu_menu'].replace(menu_mapping1)

In [9]:
df = train

X = df.drop(columns=['Ilk_menu', 'Ikinci_menu', 'Ucuncu_menu'])
y = df[['Ilk_menu', 'Ikinci_menu', 'Ucuncu_menu']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

classifiers = {
    'Ridge Classifier': RidgeClassifier()
}

def jaccard_similarity_score(y_true, y_pred):
    jaccard_scores = []
    for i in range(len(y_true)):
        intersection = np.sum(np.logical_and(y_true[i], y_pred[i]))
        union = np.sum(np.logical_or(y_true[i], y_pred[i]))
        jaccard_score = intersection / union
        jaccard_scores.append(jaccard_score)
    return np.mean(jaccard_scores)

import numpy as np

def process_predictions(y_pred, y_test):
    y_test = np.array(y_test)

    def to_binary_representation(data):
        joined_array = np.array([', '.join(map(str, row)) for row in data])
        all_menus = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
        binary_rep = [[1 if menu in entry else 0 for menu in all_menus] for entry in joined_array]
        return np.array(binary_rep)

    y_pred_binary = to_binary_representation(y_pred)
    y_test_binary = to_binary_representation(y_test)

    return y_test_binary, y_pred_binary

y_pred = None
for name, clf in classifiers.items():
    print(f"Evaluating {name}...")

    model = MultiOutputClassifier(clf)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    y_true, y_pred = process_predictions(y_pred, y_test.values)

    score = jaccard_similarity_score(y_true, y_pred)
    print(f"Jaccard Score for {name}: {score}")

Evaluating Ridge Classifier...
Jaccard Score for Ridge Classifier: 0.5506264398369661


In [14]:
X_train

,month,n_seconds_1,n_seconds_2,n_seconds_3,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,...,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,carrier_encoded,devicebrand_encoded
25787,11,467.913,310.153,277.905,-1.681805,2.142508,0.562797,-1.161826,3.215327,2.680432,...,-1.328885,0.909848,0.483826,0.692989,-1.503813,-2.185377,31,48,14,0
72740,12,310.811,280.096,273.160,-1.790492,2.127541,1.071017,-1.208557,3.476660,2.698405,...,-1.744886,1.159778,-0.126202,1.842414,-1.056161,-3.069718,24,62,11,0
39526,11,201.131,145.928,47.916,-2.761982,0.012119,-0.135885,-1.355689,4.813400,2.741259,...,-2.462132,-0.433202,0.298892,-0.094245,-0.054689,-2.110920,28,52,14,18
45165,11,132.749,118.371,114.088,-1.738869,1.273165,0.061282,0.667869,5.217914,2.270354,...,-3.262526,-0.448198,-0.147357,0.844104,-0.788386,-2.958128,19,41,10,18
19376,11,758.930,288.933,186.044,-2.223320,1.023573,1.663910,0.056099,3.423272,2.460579,...,-0.156061,0.754761,0.757072,0.778722,-1.599712,-3.744726,5,29,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,10,307.836,251.927,118.365,1.238334,1.737300,-0.084119,-0.398327,2.958013,3.389990,...,-1.223947,0.469833,0.447822,2.047022,-0.560315,-2.630362,14,72,6,18
54886,12,851.542,130.979,85.147,-1.532209,1.846237,1.084361,-1.060927,3.079069,1.763055,...,-1.890342,2.128438,-0.115285,0.907422,-0.430353,-2.578184,22,37,11,16
76820,12,258.191,104.041,89.864,-1.862614,1.710194,0.050358,-1.470732,3.468172,3.248292,...,-1.012129,1.624769,1.428957,1.115229,-1.105424,-1.774638,36,57,10,0
860,10,857.067,287.388,279.076,-2.429906,1.634941,0.988270,-1.836210,2.790268,1.746827,...,-0.467326,-0.648443,1.336665,1.869787,1.095616,-2.203076,31,69,10,18


In [13]:
y_train

,Ilk_menu,Ikinci_menu,Ucuncu_menu
25787,6,4,1
72740,6,2,8
39526,9,2,4
45165,6,2,5
19376,6,2,5
...,...,...,...
6265,6,2,1
54886,6,2,4
76820,6,2,8
860,9,8,4


In [11]:
top3_indices = np.argsort(y_pred, axis=1)[:, -3:]
result = np.zeros_like(y_pred)
result[np.arange(result.shape[0])[:, np.newaxis], top3_indices] = 1

correct_rows = np.all(result == y_test, axis=1)
accuracy = np.sum(correct_rows) / len(correct_rows)
print("Accuracy:", accuracy)

<ipython-input-11-ee1d92a67157>:5: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  correct_rows = np.all(result == y_test, axis=1)


ValueError: ignored